In [1]:
import pandas as pd
import numpy as np
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV
from surprise import SVD
from surprise import KNNBaseline
from surprise import accuracy

In [2]:
df = pd.read_csv("predictions_df_29.csv")

In [3]:
df

,Unnamed: 0,uid,iid,r_ui,est,details
0,0,350494,754,1.0,0.646738,{'was_impossible': False}
1,1,363890,1430,1.0,0.012447,{'was_impossible': False}
2,2,363889,1430,1.0,0.012447,{'was_impossible': False}
3,3,363883,1430,1.0,0.012447,{'was_impossible': False}
4,4,363882,1430,1.0,0.012447,{'was_impossible': False}
...,...,...,...,...,...,...
72911,72911,2536230,324,1.0,0.117374,{'was_impossible': False}
72912,72912,2520968,324,1.0,0.117374,{'was_impossible': False}
72913,72913,2516709,325,-1.0,0.105297,{'was_impossible': False}
72914,72914,2521966,325,-1.0,0.105297,{'was_impossible': False}


In [4]:
r = list(df['r_ui'])
est = list(df['est'])

pred_df = pd.DataFrame.from_dict({'r':r, 'est':est})

In [5]:
pred_df

,r,est
0,1.0,0.646738
1,1.0,0.012447
2,1.0,0.012447
3,1.0,0.012447
4,1.0,0.012447
...,...,...
72911,1.0,0.117374
72912,1.0,0.117374
72913,-1.0,0.105297
72914,-1.0,0.105297


In [6]:
def prueba(valor, limite):
    if valor >= limite:
        return 3
    else:
        return 0

In [7]:
import numpy as np

values = pd.DataFrame()
for th_ in np.arange(-0.9, 1, 0.1):
    th_ = np.around(th_, decimals=2)
    pred_df['th_'] = pred_df['est'].apply(prueba, args=[th_])
    pred_df['tn'] = pred_df['r'] + pred_df['th_']
    x = pred_df['tn'].value_counts()
    values[(th_)] = x



In [8]:
pred_df['th_'] = pred_df['est'].apply(prueba, args=[th_])

In [9]:
pred_df['tn'] = pred_df['r'] + pred_df['th_']

In [10]:
pred_df

,r,est,th_,tn
0,1.0,0.646738,0,1.0
1,1.0,0.012447,0,1.0
2,1.0,0.012447,0,1.0
3,1.0,0.012447,0,1.0
4,1.0,0.012447,0,1.0
...,...,...,...,...
72911,1.0,0.117374,0,1.0
72912,1.0,0.117374,0,1.0
72913,-1.0,0.105297,0,-1.0
72914,-1.0,0.105297,0,-1.0


In [11]:
values

,-0.9,-0.8,-0.7,-0.6,-0.5,-0.4,-0.3,-0.2,-0.1,-0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
4.0,35866,35809,35720,35559,35387,35105,34370,33899,33385,32211,27807,20073,12829,10051,8438,7110,5664,4227,2740
2.0,34345,33296,32217,31199,30393,29097,26734,25431,18904,17918,13343,8004,3792,2473,1214,738,537,367,231
-1.0,2592,3641,4720,5738,6544,7840,10203,11506,18033,19019,23594,28933,33145,34464,35723,36199,36400,36570,36706
1.0,113,170,259,420,592,874,1609,2080,2594,3768,8172,15906,23150,25928,27541,28869,30315,31752,33239


In [12]:
val_tp = values.T
val_tp

,4.0,2.0,-1.0,1.0
-0.9,35866,34345,2592,113
-0.8,35809,33296,3641,170
-0.7,35720,32217,4720,259
-0.6,35559,31199,5738,420
-0.5,35387,30393,6544,592
-0.4,35105,29097,7840,874
-0.3,34370,26734,10203,1609
-0.2,33899,25431,11506,2080
-0.1,33385,18904,18033,2594
-0.0,32211,17918,19019,3768


In [13]:
#-1.0 = Negativos
#5.0 = TP
#2.0 = Positivos


In [14]:
# Precisión: TP/TP+FP
val_tp['REC'] = val_tp[4.0] / (val_tp[4.0] + val_tp[2.0])
# Sencibilidad: TP/Positivos
val_tp['TPR'] = val_tp[4.0] / pred_df['r'].value_counts()[1]
# FP rate: fp/n
val_tp['FPR'] = val_tp[4.0] / pred_df['r'].value_counts()[-1]
# Accuracy: (TP+TN)/(TP+FP+FN+TN)
val_tp['Accuracy'] = (val_tp[4.0] + val_tp[-1.0]) / (pred_df['r'].value_counts()[1] + pred_df['r'].value_counts()[-1])


In [15]:
val_tp

,4.0,2.0,-1.0,1.0,REC,TPR,FPR,Accuracy
-0.9,35866,34345,2592,113,0.510832,0.996859,0.971005,0.527429
-0.8,35809,33296,3641,170,0.518182,0.995275,0.969462,0.541034
-0.7,35720,32217,4720,259,0.525781,0.992801,0.967052,0.554611
-0.6,35559,31199,5738,420,0.532655,0.988327,0.962693,0.566364
-0.5,35387,30393,6544,592,0.537960,0.983546,0.958037,0.575059
-0.4,35105,29097,7840,874,0.546790,0.975708,0.950402,0.588965
-0.3,34370,26734,10203,1609,0.562484,0.955279,0.930503,0.611292
-0.2,33899,25431,11506,2080,0.571364,0.942188,0.917752,0.622703
-0.1,33385,18904,18033,2594,0.638471,0.927902,0.903836,0.705168
-0.0,32211,17918,19019,3768,0.642562,0.895272,0.872052,0.702589


In [16]:
val_tp.to_csv('pruebas_-1_1v3.csv',header=True,index=True)